# Step 1: Data Preparation

In [1]:

import os
import shutil

# Define paths to your dataset folders
dataset_dir = r'D:\My_learning\CNN_Project\Driver_atention\Data\train'
train_dir = r'train_data'
val_dir = r'test_data'

# Ensure the directory structure for train and validation sets
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Create train/validation set with class representation maintained
for class_name in ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']:
    class_path = os.path.join(dataset_dir, class_name)
    train_class_path = os.path.join(train_dir, class_name)
    val_class_path = os.path.join(val_dir, class_name)
    os.makedirs(train_class_path, exist_ok=True)
    os.makedirs(val_class_path, exist_ok=True)
    
    class_files = os.listdir(class_path)
    num_train_samples = int(len(class_files) * 0.8)  # 80% for training, 20% for validation
    train_files = class_files[:num_train_samples]
    val_files = class_files[num_train_samples:]
    
    for file in train_files:
        shutil.copy(os.path.join(class_path, file), os.path.join(train_class_path, file))
    
    for file in val_files:
        shutil.copy(os.path.join(class_path, file), os.path.join(val_class_path, file))

In [2]:
# BAsic info of images

In [3]:
from PIL import Image

# Replace 'image_path.jpg' with the actual path to your image file.
image_path = r'D:\My_learning\CNN_Project\Driver_atention\Data\train\c0\img_34.jpg'

# Open the image using PIL
try:
    img = Image.open(image_path)
    
    # Display basic information about the image
    print("Image format:", img.format)
    print("Image size:", img.size)
    print("Image mode:", img.mode)
    
    # Do further processing or display the image as needed
    
except FileNotFoundError:
    print("Image file not found.")
except Exception as e:
    print("An error occurred while processing the image:", e)

Image format: JPEG
Image size: (640, 480)
Image mode: RGB


In [4]:
import numpy as np
imag_arr=np.array(img)

In [5]:
imag_arr.min(),imag_arr.max(),

(0, 255)

# 2) IMAGE PREPROCESSING

In [6]:
# !pip install tensorflow

In [7]:
import tensorflow as tf

In [8]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Constants
img_height, img_width = 640, 480
num_classes = 10
batch_size = 32
epochs = 10

# Paths
train_dir = r'train_data'
val_dir = r'test_data'
model_save_path = r'model/model.h5'

# Data Augmentation for Training Set
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Validation Data Generator
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load and Prepare Training Data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load and Prepare Validation Data
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)



Found 17934 images belonging to 10 classes.
Found 4490 images belonging to 10 classes.


In [9]:
# Model Creation
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])



In [10]:
# Compile the model
model.compile(optimizer=optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [11]:

# Model Training
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)
                                                

Epoch 1/10
560/560 [==============================] - 2895s 5s/step - loss: 2.1745 - accuracy: 0.2759 - val_loss: 1.1170 - val_accuracy: 0.6654
Epoch 2/10
560/560 [==============================] - 2952s 5s/step - loss: 1.0897 - accuracy: 0.6369 - val_loss: 0.5270 - val_accuracy: 0.8629
Epoch 3/10
560/560 [==============================] - 3504s 6s/step - loss: 0.7249 - accuracy: 0.7590 - val_loss: 0.2571 - val_accuracy: 0.9259
Epoch 4/10
560/560 [==============================] - 3276s 6s/step - loss: 0.5452 - accuracy: 0.8226 - val_loss: 0.1936 - val_accuracy: 0.9507
Epoch 5/10
560/560 [==============================] - 3140s 6s/step - loss: 0.4428 - accuracy: 0.8571 - val_loss: 0.1405 - val_accuracy: 0.9676
Epoch 6/10
560/560 [==============================] - 3174s 6s/step - loss: 0.3749 - accuracy: 0.8767 - val_loss: 0.1252 - val_accuracy: 0.9670
Epoch 7/10
560/560 [==============================] - 3118s 6s/step - loss: 0.3207 - accuracy: 0.8984 - val_loss: 0.1206 - val_accuracy:

In [12]:

# Save the trained model
model.save('Driver_attention_model.h5')

print("Model training and saving complete.")

Model training and saving complete.


### Model validation

In [14]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix

# Path to the directory where you split your dataset
base_dir = 'path/to/split/dataset'

# Path to the saved trained model
model_path = 'Driver_attention_model.h5'

# Parameters
batch_size = 32

# Load the trained model
model = load_model(model_path)

# Create data generators for validation set
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(
    os.path.join('D:\My_learning\CNN_Project\Driver_atention', 'test_data'),
    target_size=(img_height, img_width),  # Adjust based on your model input size
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Get true labels from the generator
true_labels = validation_generator.classes

# Generate predictions using the model
predictions = model.predict(validation_generator, steps=len(validation_generator), verbose=1)
predicted_labels = np.argmax(predictions, axis=1)

# Calculate metrics
accuracy = np.sum(predicted_labels == true_labels) / len(true_labels)
report = classification_report(true_labels, predicted_labels, target_names=validation_generator.class_indices.keys())
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Print results
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

Found 4490 images belonging to 10 classes.
141/141 [==============================] - 153s 1s/step
Accuracy: 0.9788
Classification Report:
               precision    recall  f1-score   support

          c0       0.99      0.98      0.99       498
          c1       0.98      1.00      0.99       454
          c2       1.00      0.98      0.99       464
          c3       0.99      0.99      0.99       470
          c4       0.98      0.99      0.98       466
          c5       1.00      0.98      0.99       463
          c6       0.98      0.97      0.98       465
          c7       0.99      0.99      0.99       401
          c8       0.90      0.96      0.93       383
          c9       0.98      0.95      0.96       426

    accuracy                           0.98      4490
   macro avg       0.98      0.98      0.98      4490
weighted avg       0.98      0.98      0.98      4490

Confusion Matrix:
 [[488   1   0   1   2   0   0   0   0   6]
 [  0 453   0   1   0   0   0   0   0  

### Prediction on the test Out of tiem data 

In [15]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing import image
from keras.models import load_model

# Path to the directory containing the images you want to predict
image_dir = 'D:\My_learning\CNN_Project\Driver_atention\Data\OOT'

# Path to the saved trained model
model_path = r'D:\My_learning\CNN_Project\Driver_atention\Driver_attention_model.h5'

# Load the trained model
model = load_model(model_path)

# List of class labels
class_labels = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

# Initialize lists to store image file names and prediction values
file_names = []
predictions = []

# Iterate over the images in the directory
for img_file in os.listdir(image_dir):
    img_path = os.path.join(image_dir, img_file)
    img = image.load_img(img_path, target_size=(img_height, img_width))  # Adjust based on your model input size
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.0  # Normalize pixel values

    # Make predictions
    prediction = model.predict(img)
    predicted_class = class_labels[np.argmax(prediction)]

    # Append image file name and prediction value
    file_names.append(img_file)
    predictions.append(predicted_class)

# Create a DataFrame to store the results
results_df = pd.DataFrame({'Image File': file_names, 'Prediction': predictions})

# Write the DataFrame to an Excel file
output_excel = 'predictions.xlsx'
results_df.to_excel(output_excel, index=False)

print(f"Predictions saved to {output_excel}")

Predictions saved to predictions.xlsx
